In [ ]:
#%matplotlib inline
#%pylab inline

In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

#docstrings of the different self-made classes within the self-made module
#cdoc = calc.__doc__
#ddoc = det.__doc__
#rdoc = rf.__doc__



day = 150926
detector = det.n5
data_type = 'ctime'
year = int('20' + str(day)[0:2])

#get the iso-date-format from the day
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))

#get the ordinal indicator for the date
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

#read the measurement data
ctime_data = rf.ctime(detector, day)
echan = ctime_data[0]
total_counts = ctime_data[1]
echan_counts = ctime_data[2]
total_rate = ctime_data[3]
echan_rate = ctime_data[4]
bin_time = ctime_data[5]
good_time = ctime_data[6]
exptime = ctime_data[7]
bin_time_mid = np.array((bin_time[:,0]+bin_time[:,1])/2)

total_rate = np.sum(echan_rate[1:-2], axis = 0)

echan = 2 #define which energy-channels one wants to look at
counts = echan_rate[echan]
#counts = total_rate

#read the satellite data
sat_data = rf.poshist_bin(day, bin_time_mid, detector, data_type)
sat_time_bin = sat_data[0]
sat_pos_bin = sat_data[1]
sat_lat_bin = sat_data[2]
sat_lon_bin = sat_data[3]
sat_q_bin = sat_data[4]

#calculate the sun data
sun_data = calc.sun_ang_bin(detector, day, bin_time_mid, data_type)
sun_ang_bin = sun_data[0]
sun_ang_bin = calc.ang_eff(sun_ang_bin, echan)[0]
sun_rad = sun_data[2]
sun_ra = sun_rad[:,0]
sun_dec = sun_rad[:,1]
sun_occ = calc.src_occultation_bin(day, sun_ra, sun_dec, bin_time_mid)[0]

#calculate the earth data
earth_data = calc.earth_ang_bin(detector, day, bin_time_mid, data_type)
earth_ang_bin = earth_data[0]
#earth_ang_bin = calc.ang_eff(earth_ang_bin, echan)[0]
earth_ang_bin = calc.earth_occ_eff(earth_ang_bin, echan)

#read the SFL data
flares = rf.flares(year)
flares_day = flares[0]
flares_time = flares[1]
if np.any(flares_day == day) == True:
    flares_today = flares_time[:,np.where(flares_day == day)]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
else:
    flares_today = np.array(-5)

#read the mcilwain parameter data
sat_time = rf.poshist(day)[0]
lat_data = rf.mcilwain(day)
mc_b = lat_data[1]
mc_l = lat_data[2]

mc_b = calc.intpol(mc_b, day, 0, sat_time, bin_time_mid)[0]
mc_l = calc.intpol(mc_l, day, 0, sat_time, bin_time_mid)[0]

magnetic = mc_l
magnetic = magnetic - np.mean(magnetic, dtype=np.float64)

#constant function corresponding to the diffuse y-ray background
cgb = np.ones(len(total_rate))

#counts[120000:] = 0
cgb[np.where(total_rate == 0)] = 0
earth_ang_bin[np.where(total_rate == 0)] = 0
sun_ang_bin[np.where(sun_occ == 0)] = 0
sun_ang_bin[np.where(total_rate == 0)] = 0
magnetic[np.where(total_rate == 0)] = 0

#remove vertical movement from scaling sun_ang_bin
sun_ang_bin[sun_ang_bin>0] = sun_ang_bin[sun_ang_bin>0] - np.min(sun_ang_bin[sun_ang_bin>0])

work_module.py:1525: RuntimeWarning: invalid value encountered in arccos
  ang_det_sun = np.arccos(scalar_product)
work_module.py:1085: RuntimeWarning: invalid value encountered in arccos
  ang_det_geo = np.arccos(scalar_product)


In [2]:
saa_exits = [0]
for i in range(1, len(total_rate)):
    if np.logical_and(total_rate[i-1] == 0, total_rate[i] != 0):
        print i
        saa_exits.append(i)
saa_exits = np.array(saa_exits)

if saa_exits[1] - saa_exits[0] < 10:
    saa_exits = np.delete(saa_exits, 0)

2399
22312
167308
185443
202562
219471
236937
255193
273998


In [23]:
#def exp_func(x, a, b, c):
#    return a*np.exp(-b*x) + c
def exp_func(x, a, b, i, addition_f):
#    if saa_exits[i] == 0:
#        addition = 0
#    elif saa_exits[i] < 200:
#        addition = 10
#    else:
#        addition = 9
    addition = 5
    starting_point = saa_exits[i] + addition + int(addition_f/0.00000000001)
    x_func = x[starting_point:] - x[starting_point]
    func = math.fabs(a)*np.exp(-math.fabs(b)*x_func)
    zeros = np.zeros(len(x))
    zeros[starting_point:] = func
    zeros[np.where(total_rate==0)] = 0
    return zeros

In [4]:
deaths = len(saa_exits)

exp = []
for i in range(0, deaths):
    exp = np.append(exp, [40., 0.001])

In [5]:
def fit_function(x, a, b, c, d, addition_f, exp1, exp2, deaths):
        this_is_it = a*cgb + b*magnetic + c*earth_ang_bin + d*sun_ang_bin
        for i in range(0, deaths):
            this_is_it = np.add(this_is_it, exp_func(x, exp1[i], exp2[i], i, addition_f))
        return this_is_it

def wrapper_fit_function(x, deaths, a, b, c, d, addition_f, *args):
    exp1 = args[::2]
    exp2 = args[1::2]
    return fit_function(x, a, b, c, d, addition_f, np.fabs(exp1), np.fabs(exp2), deaths)

x0 = np.append(np.array([1300., 20., -12., -1., 0.]), exp)
sigma = np.array((counts + 1)**(0.5))

In [24]:
fit_results = optimization.curve_fit(lambda x, a, b, c, d, addition_f, *args: wrapper_fit_function(x, deaths, a, b, c, d, addition_f, *args), bin_time_mid, counts, x0, sigma, maxfev = 100000)
coeff = fit_results[0]

print 'CGB coefficient:',coeff[0]
print 'Magnetic field coefficient:',coeff[1]
print 'Earth angle coefficient:',coeff[2]
print 'Sun angle coefficient:',coeff[3]
print 'Addition: ', 5 + int(coeff[4]/0.00000000001)
print 'starting exp:',coeff[5],'&',coeff[6]
print 'first SAA:',coeff[7],'&',coeff[8]

CGB coefficient: 246.563609831
Magnetic field coefficient: 36.2322841925
Earth angle coefficient: -144.278987328
Sun angle coefficient: 0.189228179777
Addition:  -701
starting exp: 12.4901726809 & 0.0247321796514
first SAA: -7.27220646886 & -0.000351937560666


2.2204460492503131e-16

In [25]:
a = coeff[0]
b = coeff[1]
c = coeff[2]
d = coeff[3]
addition_f = coeff[4]
exp1 = coeff[5::2]
exp2 = coeff[6::2]

fit_curve = fit_function(bin_time_mid, a, b, c, d, addition_f, exp1, exp2, deaths)

In [ ]:
exp_func(bin_time_mid, exp01, exp02, 0)

In [ ]:
plt.plot(bin_time_mid - bin_time_mid[0], counts, 'b-')
plt.plot(bin_time_mid - bin_time_mid[0], fit_curve, 'r-')

plt.show()

In [ ]:
residual_curve = counts - fit_curve
plt.plot(bin_time_mid - bin_time_mid[0], residual_curve, 'b-')

In [26]:
#####plot-algorhythm#####
#convert the x-axis into hours of the day
plot_time_bin_date = calc.met_to_date(bin_time_mid)[0]
plot_time_bin = (plot_time_bin_date - calc.day_to_met(day)[1])*24#Time of day in hours
plot_time_sat_date = calc.met_to_date(sat_time_bin)[0]
plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours


###plot each on the same axis as converted to counts###
fig, ax1 = plt.subplots()

plot1 = ax1.plot(plot_time_bin, counts, 'b-', label = 'Countrate')
plot2 = ax1.plot(plot_time_bin, fit_curve, 'r-', label = 'Fit')
plot3 = ax1.plot(plot_time_sat, d*sun_ang_bin, 'y-', label = 'Sun angle')
plot4 = ax1.plot(plot_time_sat, c*earth_ang_bin, 'c-', label = 'Earth angle')
plot5 = ax1.plot(plot_time_sat, b*magnetic, 'g-', label = 'Magnetic field')
plot6 = ax1.plot(plot_time_sat, a*cgb, 'b--', label = 'Cosmic y-ray background')
#plot7 = ax1.plot(plot_time_sat, j2000_orb, 'y--', label = 'J2000 orbit')
#plot8 = ax1.plot(plot_time_sat, geo_orb, 'g--', label = 'Geographical orbit')

#plot vertical lines for the solar flares of the day
if np.all(flares_today != -5):
    if len(flares_today[0]) > 1:
        for i in range(0, len(flares_today[0])):
            plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
            plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
    else:
        plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
        plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')

plots = plot1 + plot2 + plot3 + plot4 + plot5 + plot6
labels = [l.get_label() for l in plots]
ax1.legend(plots, labels, loc=1)

ax1.grid()

ax1.set_xlabel('Time of day in 24h')
ax1.set_ylabel('Countrate')

#ax1.set_xlim([9.84, 9.85])
ax1.set_xlim([-0.5, 24.5])
ax1.set_ylim([-500, 1500])

plt.title(data_type + '-countrate-fit of the ' + detector.__name__ + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year))

plt.show()

#savefig("/home/tkili/Work/saa_plot.pdf",bbox_inches="tight")

In [9]:
###plot residual noise of the fitting algorithm###
plt.plot(plot_time_bin, counts - fit_curve, 'b-')

plt.xlabel('Time of day in 24h')
plt.ylabel('Residual noise')

plt.grid()

plt.title(data_type + '-counts-fit residuals of the ' + detector.__name__ + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year))

plt.ylim([-600, 600])

plt.show()

#savefig("/home/tkili/Work/saa_plot.pdf",bbox_inches="tight")